# Actividad kNN y evaluación

Para esta actividad buscaremos el mejor valor de k para un problema de clasificación de tumores de cancer. Además, compararemos la curva de entrenamiento y test para los valores de k. 


In [ ]:
#Instalando las librerias requeridas
#import sys
#!{sys.executable} -m pip install numpy, pandas, plotnine, sklearn

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

## Data
La data corresponde a un estudio de imagenes para predecir cancer de mama. Contiene 568 datos, con 30 *features* (atributos) numéricos, y 2 clases: *Maligno* (1) y *Benigno* (0)

In [ ]:
from sklearn.datasets import load_breast_cancer
loadData = load_breast_cancer(as_frame=True)
print(loadData['DESCR'])

Carguemos la data, y reemplacemos algunos nombres de columna.

In [ ]:
data = loadData['data']
data.columns = data.columns.str.replace(' ','_')
target = loadData['target']
target = 1-target # para que "1" sea "Maligno"

In [ ]:
data.describe()

Como vamos a usar KNN, es importante normalizar los datos para que la "distancia" sea comparable entre los distintos atributos. Esto lo hacemos con un `StandardScaler` para dejarlos con media 0 y varianza 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataNorm = scaler.fit_transform(data)

Finalmente, separamos los datos en un set de entrenamiento (`X_train` y `y_train`) para calibrar el modelo, y un set de test (`X_test`, `y_test`) para evaluarlo.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataNorm, target, random_state=0)


## Parte 1:  Entrenar un modelo kNN y evaluar su performance.


Genere un modelo de KNN y calibrelo para los datos de entrenamiento. Escoja el número de vecinos del modelo (más adelante, calibraremos ese parámetro).  Con esto haga una predicción para los datos de `X_test`, y guarde estos resultados en la variable `prediccion`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors= <DEFINA EL NUMERO DE VECINOS A USAR>)
KNN=KNN.fit(<Datos entrenamiento>, <Target entrenamiento>)
prediccion=KNN.predict(<Datos test>)

Evalue el performance de su modelo, para las distintas métricas. Para esto, recuerde comparar los resultados reales `y_test` con los resultados predecidos `prediccion`.

Recuerde que puede usar distintas herramientas, por ejemplo:
- Accuracy : `mt.accuracy_score(<reales>,<prediccion>)` 
- Precision: `mt.precision_score(<reales>,<prediccion>)` 
- Recall: `mt.recall_score(<reales>,<prediccion>)` 
- F1: `mt.f1_score(<reales>,<prediccion>)` 
- Reporte general: `mt.classification_report(<reales>,<prediccion>)`
- Matriz confusión: `mt.confusion_matrix(<reales>,<prediccion>)`
- Matriz confusión (plot): `mt.ConfusionMatrixDisplay(mt.confusion_matrix(<reales>,<prediccion>)).plot()`

Recuerde F1= 2TP/(2TP+FN+FP), recall= TP/(TP+FP) y precision= TP/(TP+FN)

In [ ]:
from sklearn import metrics as mt
# Escriba aquí sus evaluaciones



**Pregunta**: ¿Es buena la predicción obtenida? ¿Qué métrica de evaluación (accuracy, precision, recall, f1) deberíamos mirar? Escoga una de estas para la siguiente parte.

## Parte 2 : Repetir la evaluación usando cross-validation

Para major certeza de lo obtenido, haremos un `KFold` (sugerencia: usar 10-fold), y usaremos `cross_validate` para evaluar cada una de las métricas, reportando su media y desviación estandar.  Para esto usamos los datos completos (o sea, no `X_train`, ...) ya que `cross_validate` se encargará de separar el conjunto train y test.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.model_selection import cross_validate


#Creamos un objeto de la clase KFold 
kf = KFold(n_splits= <NUMERO DE FOLDS ELEGIDO> )

    
#Cree un objeto de la clase KNeighborsClassifier. No necesita hacer .fit, cross_validate se encarga de eso
KNN = KNeighborsClassifier(n_neighbors= <DEFINA EL NUMERO DE VECINOS A USAR>)

#Hacemos cross_validate para las distintas métricas. 
#recuerde que las métricas son 'accuracy', 'precision', 'recall', 'f1'
res = cross_validate(KNN, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=<METRICA>, cv=kf, return_train_score=True)    
print ('Score <METRICA> train:', res['train_score'].mean(),'+-',res['train_score'].std())
print ('Score <METRICA> test:', res['test_score'].mean(),'+-',res['test_score'].std())

# Repita este código para las distintas métricas


**Pregunta**: ¿Se mantienen similares a las vistas anteriormente?

## Parte 3 : Calibrar el modelo para distintos números de vecinos `n_neighbors` usando cross-validation

In [ ]:
#Creando el objeto con sus características 
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt


# Defina el número máximo de vecinos a explorar
maxK= <SU NUMERO MAXIMO DE VECINOS>

#Generando los k-fold
kf = KFold(n_splits=10)

# guardaremos los resultados acá
results = pd.DataFrame(columns=['k','Tipo', 'Metrica', 'ScoreMean', 'ScoreStd'])

for k in range(1,maxK+1): #Ciclo for para buscar el mejor valor de K
    #Cree un objeto de la clase KNeighborsClassifier con n_neighbors=k
    KNN = KNeighborsClassifier(n_neighbors=k)
    #Hacemos un ciclo for para las distintas métricas
    for metrica in ['accuracy', 'recall', 'precision', 'f1']:
        res = cross_validate(KNN, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=metrica, cv=kf, return_train_score=True)    
        results.loc[len(results)] = [k,'train',metrica,res['train_score'].mean(),res['train_score'].std()]
        results.loc[len(results)] = [k,'test',metrica,res['test_score'].mean(),res['test_score'].std()]

results

Si todo resultó bien, grafique los resultados con el siguiente código:

In [ ]:
ggplot(results, aes(x='k', color='Tipo'))\
+ geom_line(aes(y='ScoreMean', group='Tipo'))\
+ geom_errorbar(aes(ymin='ScoreMean-ScoreStd', ymax='ScoreMean+ScoreStd'))\
+ facet_wrap('Metrica')\
+ theme_bw()

**Pregunta**: ¿Cuál es el mejor número de vecinos a su parecer?  Escoga uno como "el mejor"

## Parte 4 (opcional, si tiene tiempo)
## Evaluación con curva de ROC

A partir del modelo construido, podemos evaluar si hay una mejor "probabilidad" que usar para definir si es maligno o benigno, utilizando la curva de ROC.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.model_selection import train_test_split

#Separemos los datos nuevamente en test y train
X_train, X_test, y_train, y_test = train_test_split(dataNorm, target)

#Cree un objeto de la clase KNeighborsClassifier con el número de vecinos correspondiente y entrenelo con X_train
KNNmodel = KNeighborsClassifier(n_neighbors=<SU NUMERO DE VECINOS ESCOGIDO>)
KNNmodel.fit(X_train, y_train)

#Utilice la función predict_proba del modemlo entrenado para predecir los valores de test
prediccion_prob=KNNmodel.predict_proba(X_test)

#Llamamos a la función mt.roc_curve entregue como parametro los valores reales de test y las predicciones
fpr, tpr, thresholds = mt.roc_curve(y_test, prediccion_prob[:,1])

tempDF=pd.DataFrame(zip(fpr,tpr),columns=["fpr","tpr"])
#Curva ROC
print(ggplot(tempDF)+aes(x="fpr",y="tpr")+geom_line(color="red",size=2)+
    geom_abline(color='blue') +
    theme_bw() 
)


Calculemos los valores de g-means para los valores de fpr y tpr. Recuerde gmeans es la raiz cuadrada de los tpr por 1 menos los fpr

In [ ]:
gmeans = np.sqrt(tpr * (1-fpr))
index= np.argmax(gmeans)

#Veamos el threshold encontrado
selThreshold = thresholds[index]
print('Threshold=%f, Best Threshold=%f, G-Mean=%.3f' % (index, thresholds[index], gmeans[index]))


Calcule los scores de las distintas métricas para el umbral de probabilidad indicado.

Por ejemplo, el siguiente código entrega los scores originales (es decir, cuando `prediccion_prob` es mayor que 0.5).

Repita el código para los umbrales elegidos.  

In [ ]:
print("Valores originales: ")
print("F1-test: ",mt.f1_score(y_test,prediccion_prob[:,1]>=0.5))
print("Recall: ",mt.recall_score(y_test,prediccion_prob[:,1]>=0.5))
print("Precision: ",mt.precision_score(y_test,prediccion_prob[:,1]>=0.5))
print("Accuracy: ",mt.accuracy_score(y_test,prediccion_prob[:,1]>=0.5))



In [ ]:
print("\nValores nuevo Threshold: ")


**Pregunta**: ¿Mejora el performance del modelo con este nuevo umbral?